### Imports & Setting

In [ ]:
## packages imports
library(forecast) #ma
library(TTR) #sma
library(lmtest) #dwtest
library(ggplot2)

## setting
options(repr.plot.width = 15, repr.plot.height = 8)
setwd("~/TS2024/data/Time Series Data/")

### 1번 문제

In [ ]:
## load data
export = scan("export.txt")

## time series plot : total
plot.ts(export, type = 'b', col = 'darkorange', lwd = 2, pch = 16,
        main = "수출 시계열 그림", cex.main = 2)
abline(v = seq(1, 100, by = 12), lty = 2)

##-----(1) 추세 분석을 이용한 분해-----
## trend
z = ts(export, frequency = 12)
t = 1:length(z)
trend_model = lm(z~t+I(t^2))
That = fitted(trend_model)

## seasonal
adj_trend = z - That
s = factor(cycle(z))
season_model = lm(adj_trend~0+s) ## 절편이 0인 모형
Shat = fitted(season_model)

## irregular
Ihat = z - That - Shat

## time series plots : each components
layout(matrix(c(1,1,2,3), 2, 2, byrow = TRUE)) ## 2 by 2 matrix에 위치를 찍어줌.
plot.ts(That, lwd = 2, col = 'darkorange', main="추세성분 시계열 그림", cex.main = 1.5, ylab = expression(hat(T)[t]))
plot.ts(Shat, lwd = 1.5, col = 'steelblue', main="계절성분 시계열 그림", cex.main = 1.5, ylab = expression(hat(S)[t]))
plot.ts(as.vector(Ihat), lwd = 1.5, col = 'red', main="불규칙성분 시계열 그림", cex.main = 1.5, ylab = expression(hat(I)[t]))
abline(h = 0, lty = 2)

## (2) irregular term analysis
t.test(Ihat) ## 귀무가설 기각 못함
dwtest(lm(Ihat~1), alternative = "two.sided") ## 귀무가설 기각

##-----(3) 이동평균을 이용한 분해-----
## trend
trend = ma(z, 12)

## seasonal
adj_trend = z - trend
seasonal = tapply(adj_trend, cycle(adj_trend), function(y) mean(y, na.rm = T))
seasonal = seasonal - mean(seasonal) ## 계절성분의 평균을 0으로 조정
St = ts(rep(seasonal, 10)[1:86], frequency = 12)
                  
## irregular
irregular = z - trend - St

## time series plots : each components
layout(matrix(c(1,1,2,3), 2, 2, byrow = TRUE)) ## 2 by 2 matrix에 위치를 찍어줌.
a = as.vector(trend)
b = as.vector(St)
c = as.vector(irregular)
plot(a~t, type = "l", lwd = 2, col = 'darkorange', main="추세성분 시계열 그림", cex.main = 1.5, ylab = expression(hat(T)[t]), xlab = "year(12 times)")
plot(b~t, type = 'l', lwd = 1.5, col = 'steelblue', main="계절성분 시계열 그림", cex.main = 1.5, ylab = expression(hat(S)[t]), xlab = "year(12 times)")
plot(c~t, type = 'l', lwd = 1.5, col = 'red', main="불규칙성분 시계열 그림", cex.main = 1.5, ylab = expression(hat(I)[t]), xlab = "year(12 times)")
abline(h = 0, lty = 2)

## (4) irregular term analysis
t.test(irregular) ## 귀무가설 기각 못함
dwtest(lm(irregular~1), alternative = "two.sided") ## 귀무가설 기각 못함

## (5) 1-시차 후 예측오차의 제곱합 산출
Zhat_trend = That + Shat
print(sum((z - Zhat_trend)^2)) ## 모든 점에서 예측 수행됨
Zhat_MA = trend + St
print(sum((z - Zhat_MA)^2, na.rm = TRUE)) ## 앞뒤로 예측이 없는 시점이 존재

### 2번 문제

In [ ]:
## data load
usapass = scan('usapass.txt')
z = ts(log(usapass), frequency = 12)

## time series plot : 원자료
# plot.ts(usapass, type = 'b', col = 'darkorange', lwd = 2, pch = 16, xlab = 'month', ylab = 'Number of passengers',
#         main = "미국 월별 비행기 승객 수 시계열 그림", cex.main = 2)

## time series plot : 로그 변환
plot(log(usapass), type = 'b', col = 'darkorange', lwd = 2, pch = 16, xlab = 'month', ylab = expression(log(z)),
        main = "로그 변환 시계열 그림", cex.main = 2)
abline(v = seq(1, 150, by = 12), lty = 2)

##-----(2) 계절추세모형 적합-----
## 모형 적합
t = 1:length(z)
s = factor(cycle(z))
trend_model = lm(z~t+s) ## s1를 기저로 처리
rsid = resid(trend_model)

## 실제 데이터와 각 모형에서 구해진 추정값 비교
Z_trend = fitted(trend_model)
ts.plot(z, Z_trend, lty = 1:2, col = 1:2, lwd = 2,
        ylab = expression(log(z)), main = "실제 데이터와 추세 모형 추정값의 비교")
legend('topleft', c("real", "fitted"), lty = 1:2, lwd = 2, col = 1:2)

## 잔차 분석
plot.ts(rsid, lwd = 2, col = 'red', ylab = expression(hat(e[i])), main = "추세 모형 잔차 그림")
abline(h = 0, lty = 2)
t.test(rsid)
dwtest(lm(rsid~1), alternative = "two.sided")
bptest(trend_model)

##-----(3) 계절지수평활-----
## 모형 적합
fit_holt = HoltWinters(z)
smooth_model = hw(z, alpha = fit_holt$alpha, beta = fit_holt$beta, gamma = fit_holt$gamma, initial = 'simple', h = 12)
rsid = smooth_model$residuals

## 실제 데이터와 각 모형에서 구해진 추정값 비교
Z_smooth = fitted(smooth_model)
ts.plot(z, Z_smooth, lty = 1:2, col = 1:2, lwd = 2,
        ylab = expression(log(z)), main = "실제 데이터와 계절지수평활 추정값의 비교")
legend('topleft', c("real", "fitted"), lty = 1:2, lwd = 2, col = 1:2)

## 잔차 분석
plot.ts(rsid, lwd = 2, col = 'red', ylab = expression(hat(e[i])), main = "추세 모형 잔차 그림")
abline(h = 0, lty = 2)
t.test(rsid)
dwtest(lm(rsid~1), alternative = "two.sided")
bptest(trend_model)

##-----(4) 분해법-----
## trend
decomp_trend_model = lm(z~t)
Tt = fitted(decomp_trend_model)

## seasonal
adj_trend = z - Tt
s = factor(cycle(z))
season_model = lm(adj_trend~0+s) ## 절편이 0인 모형
St = fitted(season_model)

## irregular
It = z - Tt - St

## 실제 데이터와 각 모형에서 구해진 추정값 비교
Z_decomp = Tt + St
ts.plot(z, Z_decomp, lty = 1:2, col = 1:2, lwd = 2,
        ylab = expression(log(z)), main = "실제 데이터와 분해법에서 추정값의 비교")
legend('topleft', c("real", "fitted"), lty = 1:2, lwd = 2, col = 1:2)

## time series plots : each components
layout(matrix(c(1,1,2,3), 2, 2, byrow = TRUE)) ## 2 by 2 matrix에 위치를 찍어줌.
plot.ts(Tt, lwd = 2, col = 'darkorange', main="추세성분 시계열 그림", cex.main = 1.5, ylab = expression(hat(T)[t]))
plot.ts(St, lwd = 1.5, col = 'steelblue', main="계절성분 시계열 그림", cex.main = 1.5, ylab = expression(hat(S)[t]))
plot.ts(as.vector(It), lwd = 1.5, col = 'red', main="불규칙성분 시계열 그림", cex.main = 1.5, ylab = expression(hat(I)[t]))
abline(h = 0, lty = 2)

##-----(5) SSE 비교-----
SSE_trend = sum(resid(trend_model)**2)
SSE_smooth = sum(resid(smooth_model)**2) ## 앞뒤 데이터 예측 안됨
SSE_decomp = sum((z-Z_decomp)**2)
print(SSE_trend)
print(SSE_smooth)
print(SSE_decomp)

### 3번 문제

In [ ]:
z = c(7, 6, 5, 8, 9, 4, 5, 5, 4, 6, 7, 8, 5, 6, 5)
n = length(z)
zbar = mean(z)

## sample covariance
gamma0 = sum((z - zbar)**2)/15
gamma1 = sum((z[1:14] - zbar)*(z[2:15] - zbar))/15
gamma2 = sum((z[1:13] - zbar)*(z[3:15] - zbar))/15
gamma3 = sum((z[1:12] - zbar)*(z[4:15] - zbar))/15

## SACF
rho1 = gamma1/gamma0
rho2 = gamma2/gamma0
rho3 = gamma3/gamma0

## SPACF
phi1 = rho1
phi2 = (rho2 - rho1^2)/(1-rho1^2)

## 값 산출
print(paste(gamma0, gamma1, gamma2, gamma3))
print(paste(rho1, rho2, rho3))
print(paste(phi1, phi2))

### 5번 문제